In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from difflib import SequenceMatcher

# ============ Staged Training System ============
class StagedCryptoTrainer:
    """
    3-stage training approach:
    Stage 1: Train Alice+Bob for perfect reconstruction (no Eve)
    Stage 2: Train Eve to attack (freeze Alice+Bob)
    Stage 3: Adversarial fine-tuning (all together)
    """
    def __init__(self, vocab_size=98, embed_dim=128, alice_layers=3,
                 bob_layers=3, eve_layers=4, max_len=128,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.max_len = max_len

        # Import components
        from v2_part1_simplified import StringProcessor, KeyManager
        from v2_part2_alice import AliceEncryptor
        from v2_part3_bob_eve import BobDecryptor, EveAttacker

        self.processor = StringProcessor(max_length=max_len)
        self.key_manager = KeyManager(device=device)

        # Initialize networks
        self.alice_wrapper = AliceEncryptor(vocab_size, embed_dim, alice_layers, max_len, device)
        self.bob_wrapper = BobDecryptor(vocab_size, embed_dim, bob_layers, max_len, device)
        self.eve_wrapper = EveAttacker(vocab_size, embed_dim, eve_layers, max_len, device)

        self.alice = self.alice_wrapper.alice
        self.bob = self.bob_wrapper.bob
        self.eve = self.eve_wrapper.eve

        # Optimizers
        self.opt_alice = optim.Adam(self.alice.parameters(), lr=0.003)
        self.opt_bob = optim.Adam(self.bob.parameters(), lr=0.003)
        self.opt_eve = optim.Adam(self.eve.parameters(), lr=0.0005)

        # Schedulers
        self.sched_alice = optim.lr_scheduler.ReduceLROnPlateau(self.opt_alice, patience=5, factor=0.5)
        self.sched_bob = optim.lr_scheduler.ReduceLROnPlateau(self.opt_bob, patience=5, factor=0.5)
        self.sched_eve = optim.lr_scheduler.ReduceLROnPlateau(self.opt_eve, patience=5, factor=0.8)

        # Loss
        self.ce_loss = nn.CrossEntropyLoss(ignore_index=0)

        self.current_stage = 1

    def train_stage1_reconstruction(self, messages, num_epochs=50):
        """
        Stage 1: Train Alice+Bob for perfect reconstruction
        Goal: Bob accuracy > 90%
        """
        print("\n" + "="*70)
        print("STAGE 1: TRAINING ALICE+BOB FOR RECONSTRUCTION")
        print("="*70)

        history = {'loss': [], 'accuracy': []}

        for epoch in range(num_epochs):
            # Sample batch
            batch = np.random.choice(messages, size=min(8, len(messages)), replace=False).tolist()
            tokens = self.processor.batch_encode(batch).to(self.device)

            # Generate keys
            keys = self.key_manager.generate_keys_for_batch(len(batch), tokens.size(1))

            # Forward pass
            self.opt_alice.zero_grad()
            self.opt_bob.zero_grad()

            self.alice.train()
            self.bob.train()

            encrypted = self.alice(tokens, keys['key_tensors'])
            logits = self.bob(encrypted, keys['key_tensors'])

            # Reconstruction loss
            loss = self.ce_loss(logits.reshape(-1, logits.size(-1)), tokens.reshape(-1))

            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.alice.parameters(), 1.0)
            torch.nn.utils.clip_grad_norm_(self.bob.parameters(), 1.0)

            self.opt_alice.step()
            self.opt_bob.step()

            # Calculate accuracy
            with torch.no_grad():
                pred_tokens = torch.argmax(logits, dim=-1)
                mask = tokens != 0
                correct = ((pred_tokens == tokens) & mask).sum().item()
                total = mask.sum().item()
                accuracy = correct / total if total > 0 else 0

            history['loss'].append(loss.item())
            history['accuracy'].append(accuracy)

            if epoch % 10 == 0:
                print(f"Epoch {epoch:3d} | Loss: {loss.item():.4f} | Accuracy: {accuracy*100:.1f}%")

            if epoch % 10 == 0:
                total_grad = sum(p.grad.abs().mean().item()
                              for p in self.alice.parameters() if p.grad is not None)
                print(f"Epoch {epoch:3d} | Loss: {loss.item():.4f} | "
                  f"Accuracy: {accuracy*100:.1f}% | Grad: {total_grad:.4f}")

            # Update learning rate
            self.sched_alice.step(loss)
            self.sched_bob.step(loss)

        final_acc = np.mean(history['accuracy'][-10:])
        print(f"\nStage 1 Complete! Final accuracy: {final_acc*100:.1f}%")

        if final_acc > 0.90:
            print("SUCCESS: Bob can decrypt accurately!")
            self.current_stage = 2
        else:
            print("WARNING: Bob accuracy < 90%. Consider training longer.")

        return history

    def train_stage2_adversary(self, messages, num_epochs=30):
        """
        Stage 2: Train Eve to attack (freeze Alice+Bob)
        Goal: Eve learns patterns but can't match Bob
        """
        print("\n" + "="*70)
        print("STAGE 2: TRAINING EVE (ADVERSARY)")
        print("="*70)

        # Freeze Alice and Bob
        for param in self.alice.parameters():
            param.requires_grad = False
        for param in self.bob.parameters():
            param.requires_grad = False

        history = {'loss': [], 'accuracy': []}

        for epoch in range(num_epochs):
            batch = np.random.choice(messages, size=min(4, len(messages)), replace=False).tolist()
            tokens = self.processor.batch_encode(batch).to(self.device)

            keys = self.key_manager.generate_keys_for_batch(len(batch), tokens.size(1))

            # Get encrypted data from Alice
            with torch.no_grad():
                encrypted = self.alice(tokens, keys['key_tensors'])

            # Train Eve
            self.opt_eve.zero_grad()
            self.eve.train()

            eve_logits = self.eve(encrypted)
            loss = self.ce_loss(eve_logits.reshape(-1, eve_logits.size(-1)), tokens.reshape(-1))

            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.eve.parameters(), 1.0)
            self.opt_eve.step()

            # Calculate accuracy
            with torch.no_grad():
                pred_tokens = torch.argmax(eve_logits, dim=-1)
                mask = tokens != 0
                correct = ((pred_tokens == tokens) & mask).sum().item()
                total = mask.sum().item()
                accuracy = correct / total if total > 0 else 0

            history['loss'].append(loss.item())
            history['accuracy'].append(accuracy)

            if epoch % 10 == 0:
                print(f"Epoch {epoch:3d} | Loss: {loss.item():.4f} | Eve Accuracy: {accuracy*100:.1f}%")

            self.sched_eve.step(loss)

        # Unfreeze Alice and Bob
        for param in self.alice.parameters():
            param.requires_grad = True
        for param in self.bob.parameters():
            param.requires_grad = True

        print(f"\nStage 2 Complete! Eve trained.")
        self.current_stage = 3

        return history

    def train_stage3_adversarial(self, messages, num_epochs=50):
        """
        Stage 3: Adversarial fine-tuning
        Goal: Alice learns to fool Eve while Bob still decrypts
        """
        print("\n" + "="*70)
        print("STAGE 3: ADVERSARIAL FINE-TUNING")
        print("="*70)

        history = {
            'bob_loss': [], 'eve_loss': [],
            'bob_acc': [], 'eve_acc': [], 'ratio': []
        }

        for epoch in range(num_epochs):
            batch = np.random.choice(messages, size=min(4, len(messages)), replace=False).tolist()
            tokens = self.processor.batch_encode(batch).to(self.device)
            keys = self.key_manager.generate_keys_for_batch(len(batch), tokens.size(1))

            # Train Alice+Bob
            self.opt_alice.zero_grad()
            self.opt_bob.zero_grad()

            self.alice.train()
            self.bob.train()

            encrypted = self.alice(tokens, keys['key_tensors'])
            bob_logits = self.bob(encrypted, keys['key_tensors'])
            bob_loss = self.ce_loss(bob_logits.reshape(-1, bob_logits.size(-1)), tokens.reshape(-1))

            bob_loss.backward()
            self.opt_alice.step()
            self.opt_bob.step()

            # Train Eve
            self.opt_eve.zero_grad()
            self.eve.train()

            with torch.no_grad():
                encrypted_for_eve = self.alice(tokens, keys['key_tensors'])

            eve_logits = self.eve(encrypted_for_eve)
            eve_loss = self.ce_loss(eve_logits.reshape(-1, eve_logits.size(-1)), tokens.reshape(-1))

            eve_loss.backward()
            self.opt_eve.step()

            # Adversarial update for Alice (make Eve fail)
            self.opt_alice.zero_grad()

            encrypted_adv = self.alice(tokens, keys['key_tensors'])
            eve_logits_adv = self.eve(encrypted_adv)
            adv_loss = -self.ce_loss(eve_logits_adv.reshape(-1, eve_logits_adv.size(-1)), tokens.reshape(-1))

            (adv_loss * 0.5).backward()  # Weight adversarial loss lower
            self.opt_alice.step()

            # Calculate metrics
            with torch.no_grad():
                bob_pred = torch.argmax(bob_logits, dim=-1)
                eve_pred = torch.argmax(eve_logits, dim=-1)
                mask = tokens != 0

                bob_acc = ((bob_pred == tokens) & mask).sum().item() / mask.sum().item()
                eve_acc = ((eve_pred == tokens) & mask).sum().item() / mask.sum().item()
                ratio = eve_loss.item() / (bob_loss.item() + 1e-8)

            history['bob_loss'].append(bob_loss.item())
            history['eve_loss'].append(eve_loss.item())
            history['bob_acc'].append(bob_acc)
            history['eve_acc'].append(eve_acc)
            history['ratio'].append(ratio)

            if epoch % 10 == 0:
                print(f"Epoch {epoch:3d} | Bob: {bob_loss.item():.3f} ({bob_acc*100:.1f}%) | "
                      f"Eve: {eve_loss.item():.3f} ({eve_acc*100:.1f}%) | Ratio: {ratio:.2f}x")

        print(f"\nStage 3 Complete!")
        return history

    def full_training(self, messages, stage1_epochs=50, stage2_epochs=30, stage3_epochs=50):
        """Run all 3 stages"""
        print("\n" + "="*70)
        print("FULL STAGED TRAINING")
        print(f"Dataset: {len(messages)} messages")
        print("="*70)

        h1 = self.train_stage1_reconstruction(messages, stage1_epochs)
        h2 = self.train_stage2_adversary(messages, stage2_epochs)
        h3 = self.train_stage3_adversarial(messages, stage3_epochs)

        return {'stage1': h1, 'stage2': h2, 'stage3': h3}


# ============ Evaluation ============
class CryptoEvaluator:
    @staticmethod
    def evaluate(trainer, test_messages):
        print("\n" + "="*70)
        print("FINAL EVALUATION")
        print("="*70)

        results = {'bob_sim': [], 'eve_sim': [], 'examples': []}

        for i, original in enumerate(test_messages[:5]):
            tokens = trainer.processor.encode(original).unsqueeze(0).to(trainer.device)
            keys = trainer.key_manager.generate_keys_for_batch(1, tokens.size(1))

            with torch.no_grad():
                encrypted = trainer.alice(tokens, keys['key_tensors'])
                bob_logits = trainer.bob(encrypted, keys['key_tensors'])
                eve_logits = trainer.eve(encrypted)

                bob_tokens = torch.argmax(bob_logits, dim=-1)
                eve_tokens = torch.argmax(eve_logits, dim=-1)

                bob_msg = trainer.processor.decode(bob_tokens[0])
                eve_msg = trainer.processor.decode(eve_tokens[0])

            bob_sim = SequenceMatcher(None, original, bob_msg).ratio()
            eve_sim = SequenceMatcher(None, original, eve_msg).ratio()

            results['bob_sim'].append(bob_sim)
            results['eve_sim'].append(eve_sim)
            results['examples'].append((original, bob_msg, eve_msg))

            print(f"\nExample {i+1}:")
            print(f"  Original: '{original}'")
            print(f"  Bob:      '{bob_msg}' ({bob_sim*100:.1f}%)")
            print(f"  Eve:      '{eve_msg}' ({eve_sim*100:.1f}%)")

        avg_bob = np.mean(results['bob_sim'])
        avg_eve = np.mean(results['eve_sim'])

        print("\n" + "="*70)
        print(f"Bob Similarity: {avg_bob*100:.1f}% {'✓' if avg_bob > 0.9 else '✗'}")
        print(f"Eve Similarity: {avg_eve*100:.1f}% {'✓' if avg_eve < 0.3 else '✗'}")
        print(f"Security Ratio: {avg_bob/max(avg_eve,0.01):.2f}x")
        print("="*70)

        return results


# ============ Main ============
if __name__ == "__main__":
    # Dataset
    messages = np.array([
        "Hello World!",
        "This is a test.",
        "Secret message here.",
        "Encryption works!",
        "Neural crypto system.",
        "Testing ABC 123.",
        "Quick brown fox.",
        "The lazy dog jumps."
    ])

    # Initialize
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    trainer = StagedCryptoTrainer(
        vocab_size=98,
        embed_dim=128,
        alice_layers=3,
        bob_layers=3,
        eve_layers=4,
        max_len=64,
        device=device
    )

    # Train
    history = trainer.full_training(
        messages,
        stage1_epochs=100,  # Focus on reconstruction
        stage2_epochs=30,
        stage3_epochs=50
    )

    # Evaluate
    results = CryptoEvaluator.evaluate(trainer, messages)

    print("\n✓ Training complete!")


FULL STAGED TRAINING
Dataset: 8 messages

STAGE 1: TRAINING ALICE+BOB FOR RECONSTRUCTION
Epoch   0 | Loss: 4.5794 | Accuracy: 2.6%
Epoch   0 | Loss: 4.5794 | Accuracy: 2.6% | Grad: 0.0581
Epoch  10 | Loss: 3.3354 | Accuracy: 19.1%
Epoch  10 | Loss: 3.3354 | Accuracy: 19.1% | Grad: 0.0001
Epoch  20 | Loss: 3.0362 | Accuracy: 21.1%
Epoch  20 | Loss: 3.0362 | Accuracy: 21.1% | Grad: 0.0009
Epoch  30 | Loss: 2.8609 | Accuracy: 21.1%
Epoch  30 | Loss: 2.8609 | Accuracy: 21.1% | Grad: 0.0005
Epoch  40 | Loss: 2.7977 | Accuracy: 21.7%
Epoch  40 | Loss: 2.7977 | Accuracy: 21.7% | Grad: 0.0016
Epoch  50 | Loss: 2.5056 | Accuracy: 27.0%
Epoch  50 | Loss: 2.5056 | Accuracy: 27.0% | Grad: 0.0525
Epoch  60 | Loss: 2.0624 | Accuracy: 37.5%
Epoch  60 | Loss: 2.0624 | Accuracy: 37.5% | Grad: 0.0306
Epoch  70 | Loss: 1.6802 | Accuracy: 46.1%
Epoch  70 | Loss: 1.6802 | Accuracy: 46.1% | Grad: 0.0965
Epoch  80 | Loss: 1.1277 | Accuracy: 66.4%
Epoch  80 | Loss: 1.1277 | Accuracy: 66.4% | Grad: 0.0278
Epo

In [5]:
# Test file: test_reconstruction.py
import torch
import torch.nn as nn

class MinimalAutoencoder(nn.Module):
    def __init__(self, vocab_size=98, embed_dim=32):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.encoder = nn.Linear(embed_dim, embed_dim)
        self.decoder = nn.Linear(embed_dim, vocab_size)

    def forward(self, tokens):
        x = self.embedding(tokens)
        x = torch.relu(self.encoder(x))
        logits = self.decoder(x)
        return logits

# Test
model = MinimalAutoencoder().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Very simple test
test_tokens = torch.tensor([[1, 40, 69, 76, 76, 79, 2, 0]]).cuda()  # "Hello"

for epoch in range(200):
    optimizer.zero_grad()
    logits = model(test_tokens)
    loss = criterion(logits.reshape(-1, 98), test_tokens.reshape(-1))
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        pred = torch.argmax(logits, dim=-1)
        acc = (pred == test_tokens).float().mean().item()
        print(f"Epoch {epoch}: Loss {loss.item():.3f}, Acc {acc*100:.1f}%")

Epoch 0: Loss 4.738, Acc 0.0%
Epoch 20: Loss 0.008, Acc 100.0%
Epoch 40: Loss 0.000, Acc 100.0%
Epoch 60: Loss 0.000, Acc 100.0%
Epoch 80: Loss 0.000, Acc 100.0%
Epoch 100: Loss 0.000, Acc 100.0%
Epoch 120: Loss 0.000, Acc 100.0%
Epoch 140: Loss 0.000, Acc 100.0%
Epoch 160: Loss 0.000, Acc 100.0%
Epoch 180: Loss 0.000, Acc 100.0%
